# Quantum-Inspired Optimization for Protein Folding Lattice Models

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChessEngineUS/protein-lattice-qio/blob/main/notebooks/colab_run.ipynb)

This notebook demonstrates quantum-inspired optimization algorithms for solving the protein folding problem on 2D and 3D lattice models.

**What you'll learn:**
- How the HP (Hydrophobic-Polar) model simplifies protein folding
- How to implement simulated annealing and quantum-inspired optimization
- How to visualize folding trajectories and energy landscapes
- Performance comparison between classical and quantum-inspired approaches

**Runtime:** ~3-5 minutes on Colab T4 GPU (free tier)

## Setup and Installation

First, we'll clone the repository and set up the environment.

In [ ]:
import os
import sys

try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    # Remove existing directory if it exists
    !rm -rf protein-lattice-qio
    
    # Clone repository
    !git clone https://github.com/ChessEngineUS/protein-lattice-qio.git
    
    # Change to repository directory
    os.chdir('protein-lattice-qio')
    print(f'Working directory: {os.getcwd()}')
    
    # Colab comes with most packages pre-installed, so we use what's available
    print('\n✓ Using Colab pre-installed packages (numpy, torch, matplotlib, scipy, tqdm, psutil)')
    print('✓ Setup complete!')
else:
    print('Running locally - ensure dependencies are installed')
    print('Run: pip install -r requirements.txt')

## Import Libraries and Detect Hardware

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Add src to path
sys.path.insert(0, 'src')

from lattice import SquareLattice2D, CubicLattice3D
from energy import HPEnergyFunction
from optimizer import GreedyOptimizer, SimulatedAnnealingOptimizer, QuantumInspiredOptimizer
from visualization import plot_conformation, plot_energy_trajectory
from utils import set_seed, detect_device, generate_hp_sequence

set_seed(42)
device = detect_device()
print(f'\n✓ Using device: {device}')

if device == 'cuda':
    STEPS = 5000
    print('GPU detected - using full optimization steps')
else:
    STEPS = 1000
    print('CPU detected - using reduced optimization steps')

print(f'NumPy version: {np.__version__}')

## Background: The HP Model

The **HP (Hydrophobic-Polar) model** simplifies protein folding by representing each amino acid as either:
- **H** (Hydrophobic): prefers to be buried in the protein core
- **P** (Polar): prefers to be on the surface

The protein folds on a 2D or 3D lattice to maximize H-H contacts (hydrophobic core formation).

**Energy function:**
- Each H-H contact contributes -1 to the energy
- Lower energy = better folding

**Reference:** Lau & Dill (1989), *PNAS* 86(6), 2050-2054. [DOI: 10.1073/pnas.86.6.2050](https://doi.org/10.1073/pnas.86.6.2050)

## Generate Test Sequences

In [ ]:
test_sequences = [
    generate_hp_sequence(20, h_ratio=0.5),
    generate_hp_sequence(25, h_ratio=0.55),
    generate_hp_sequence(30, h_ratio=0.6),
]

print('Generated test sequences:')
for i, seq in enumerate(test_sequences):
    h_count = seq.count('H')
    print(f'  Sequence {i+1} (length={len(seq)}): {seq}')
    print(f'    Hydrophobic ratio: {h_count}/{len(seq)} = {h_count/len(seq):.2%}')

sequence = test_sequences[1]
print(f'\n✓ Using sequence of length {len(sequence)} for detailed analysis')

## Run Optimization Algorithms

We'll compare three approaches:
1. **Greedy**: Fast but often gets stuck in local minima
2. **Simulated Annealing**: Classical optimization with temperature schedule
3. **Quantum-Inspired**: Adds tunneling moves to escape local minima

**Reference for quantum-inspired approach:** Kadowaki & Nishimori (1998), *Physical Review E* 58(5), 5355. [DOI: 10.1103/PhysRevE.58.5355](https://doi.org/10.1103/PhysRevE.58.5355)

In [ ]:
lattice = SquareLattice2D()
energy_fn = HPEnergyFunction()

optimizers = {
    'Greedy': GreedyOptimizer(device=device),
    'Simulated Annealing': SimulatedAnnealingOptimizer(
        initial_temp=10.0, final_temp=0.1, steps=STEPS, device=device
    ),
    'Quantum-Inspired': QuantumInspiredOptimizer(
        initial_temp=10.0, final_temp=0.1, steps=STEPS, tunnel_rate=0.1, device=device
    ),
}

results = {}
print('\nRunning optimizations...\n')

for name, optimizer in optimizers.items():
    print(f'Running {name}...')
    coords, energy, trajectory = optimizer.optimize(sequence, lattice, energy_fn)
    results[name] = {'coords': coords, 'energy': energy, 'trajectory': trajectory}
    print(f'  Final energy: {energy:.4f}\n')

print('✓ Optimization complete!')

## Visualize Final Conformations

Red dots = Hydrophobic (H) residues  
Blue dots = Polar (P) residues

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for ax, (name, result) in zip(axes, results.items()):
    coords = result['coords']
    energy = result['energy']
    
    ax.plot(coords[:, 0], coords[:, 1], 'k-', alpha=0.3, linewidth=2)
    
    colors = ['red' if s == 'H' else 'blue' for s in sequence]
    ax.scatter(coords[:, 0], coords[:, 1], c=colors, s=150, 
              edgecolors='black', linewidth=2, zorder=10)
    
    for i, (x, y) in enumerate(coords):
        ax.text(x, y, str(i), ha='center', va='center', 
               fontsize=7, fontweight='bold', color='white')
    
    ax.set_xlabel('X', fontsize=10)
    ax.set_ylabel('Y', fontsize=10)
    ax.set_title(f'{name}\nEnergy: {energy:.4f}', fontsize=11, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.set_aspect('equal')

plt.tight_layout()
plt.show()
print('Red = Hydrophobic (H), Blue = Polar (P)')

## Energy Convergence Trajectories

Visualize how each algorithm's energy evolves during optimization.

In [ ]:
fig = plot_energy_trajectory({name: res['trajectory'] for name, res in results.items()})
plt.show()

print('\nConvergence Statistics:')
for name, result in results.items():
    trajectory = result['trajectory']
    initial_energy = trajectory[0] if len(trajectory) > 0 else 0
    final_energy = result['energy']
    improvement = initial_energy - final_energy
    print(f'  {name}:')
    print(f'    Initial energy: {initial_energy:.4f}')
    print(f'    Final energy: {final_energy:.4f}')
    print(f'    Improvement: {improvement:.4f}')

## Algorithm Comparison Summary

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

names = list(results.keys())
energies = [results[name]['energy'] for name in names]
colors = ['#1f77b4', '#ff7f0e', '#2ca02c']

ax1.bar(names, energies, color=colors)
ax1.set_ylabel('Final Energy', fontsize=12)
ax1.set_title('Final Energy Comparison\n(Lower is Better)', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3, axis='y')

best_idx = np.argmin(energies)
ax1.bar(best_idx, energies[best_idx], color='gold', edgecolor='black', linewidth=2)

traj_lengths = [len(results[name]['trajectory']) for name in names]
ax2.bar(names, traj_lengths, color=colors)
ax2.set_ylabel('Optimization Steps', fontsize=12)
ax2.set_title('Number of Optimization Steps', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

best_algorithm = names[best_idx]
best_energy = energies[best_idx]
print(f'\n🏆 Best algorithm: {best_algorithm} with energy {best_energy:.4f}')

## Next Steps

Try experimenting with:
- Different sequence lengths and H/P ratios
- 3D cubic lattice (`CubicLattice3D()`)
- Custom energy functions
- Different temperature schedules
- Varying tunnel rates for quantum-inspired optimizer

For benchmarking on your specific hardware, run:
```bash
python scripts/benchmark.py
```